# N-gram Language Models

## Setup

In [1]:
import random

NumberOfOutputSentences = 5

## Training data

In [2]:
trainingData = """
    A cat is an animal. 
    A dog is also an animal.
    Both a cat and a dog are animals.
    Every cat is an animal. 
    Every animal is not a cat. 
    A cat is never a dog.
    The cat sat on the mat.
    The dog barks at the cat.
    The cat runs away.
"""

## Tokenization

In [3]:
tokenizedData = (
    trainingData
    .lower()
    .replace(".", " .")
    .split()
)

print(f"{tokenizedData = }")

tokenizedData = ['a', 'cat', 'is', 'an', 'animal', '.', 'a', 'dog', 'is', 'also', 'an', 'animal', '.', 'both', 'a', 'cat', 'and', 'a', 'dog', 'are', 'animals', '.', 'every', 'cat', 'is', 'an', 'animal', '.', 'every', 'animal', 'is', 'not', 'a', 'cat', '.', 'a', 'cat', 'is', 'never', 'a', 'dog', '.', 'the', 'cat', 'sat', 'on', 'the', 'mat', '.', 'the', 'dog', 'barks', 'at', 'the', 'cat', '.', 'the', 'cat', 'runs', 'away', '.']


## Calculate token frequencies
A.K.A counting the number of times each token occurs in the trainging data

In [4]:
tokenFrequencies = {}

for token in tokenizedData:
    if token in tokenFrequencies:
        tokenFrequencies[token] += 1
    else:
        tokenFrequencies[token] = 1

print(f"{tokenFrequencies = }")

tokenFrequencies = {'a': 7, 'cat': 8, 'is': 5, 'an': 3, 'animal': 4, '.': 9, 'dog': 4, 'also': 1, 'both': 1, 'and': 1, 'are': 1, 'animals': 1, 'every': 2, 'not': 1, 'never': 1, 'the': 5, 'sat': 1, 'on': 1, 'mat': 1, 'barks': 1, 'at': 1, 'runs': 1, 'away': 1}


## Print probabilities

In [5]:
for token, count in sorted(tokenFrequencies.items(), key=lambda item: item[1], reverse=True):
    print(f'{token:15} {count / len(tokenizedData) * 100:-5,.2f} %')

.               14.75 %
cat             13.11 %
a               11.48 %
is               8.20 %
the              8.20 %
animal           6.56 %
dog              6.56 %
an               4.92 %
every            3.28 %
also             1.64 %
both             1.64 %
and              1.64 %
are              1.64 %
animals          1.64 %
not              1.64 %
never            1.64 %
sat              1.64 %
on               1.64 %
mat              1.64 %
barks            1.64 %
at               1.64 %
runs             1.64 %
away             1.64 %


## Unigram model output

In [6]:
def prettyPrint(words):
    text = ''
    isTitle = True

    for word in words:
        if word != '.' and text != '':
            text += ' '

        if isTitle:
            text += word.capitalize()
        else:
            text += word

        isTitle = (word == '.')
    
    print(text)

words = []
sentenceCounter = 0

while (sentenceCounter < NumberOfOutputSentences):
    currentWord = random.choices(list(tokenFrequencies.keys()), weights=tokenFrequencies.values())[0]
    if currentWord == ".": sentenceCounter += 1
    words.append(currentWord)

prettyPrint(words)

On an animal.. Animal runs away a runs is animal dog. Animal an dog the away both. Animal every both is.


## Building a bigram model

In [7]:
bigrams = {}

for index in range(len(tokenizedData)-1):
    currentWord = tokenizedData[index]
    nextWord = tokenizedData[index + 1]

    if not currentWord in bigrams:
        bigrams[currentWord] = {nextWord: 1}
    elif nextWord not in bigrams[currentWord]:
        bigrams[currentWord][nextWord] = 1
    else: 
        bigrams[currentWord][nextWord] += 1

print(f"{bigrams = }")

bigrams = {'a': {'cat': 4, 'dog': 3}, 'cat': {'is': 3, 'and': 1, '.': 2, 'sat': 1, 'runs': 1}, 'is': {'an': 2, 'also': 1, 'not': 1, 'never': 1}, 'an': {'animal': 3}, 'animal': {'.': 3, 'is': 1}, '.': {'a': 2, 'both': 1, 'every': 2, 'the': 3}, 'dog': {'is': 1, 'are': 1, '.': 1, 'barks': 1}, 'also': {'an': 1}, 'both': {'a': 1}, 'and': {'a': 1}, 'are': {'animals': 1}, 'animals': {'.': 1}, 'every': {'cat': 1, 'animal': 1}, 'not': {'a': 1}, 'never': {'a': 1}, 'the': {'cat': 3, 'mat': 1, 'dog': 1}, 'sat': {'on': 1}, 'on': {'the': 1}, 'mat': {'.': 1}, 'barks': {'at': 1}, 'at': {'the': 1}, 'runs': {'away': 1}, 'away': {'.': 1}}


## Print probabilities

In [8]:
def print_ngram_probabilities(ngrams):
    BOLD = "\033[1m"
    RESET = "\033[0m"

    for head in sorted(ngrams.keys()):
        print(f"{BOLD}{head}{RESET}")  # Huvudord

        followers = ngrams[head]
        total = sum(followers.values())  # Totalt antal följande ord
        sorted_followers = sorted(followers.items())

        for word, count in sorted_followers:
            percent = (count / total) * 100
            print(f"    {word:<10} {percent:.2f}%")  # Procent med två decimaler
        print()

print_ngram_probabilities(bigrams)

.
    a          25.00%
    both       12.50%
    every      25.00%
    the        37.50%

a
    cat        57.14%
    dog        42.86%

also
    an         100.00%

an
    animal     100.00%

and
    a          100.00%

animal
    .          75.00%
    is         25.00%

animals
    .          100.00%

are
    animals    100.00%

at
    the        100.00%

away
    .          100.00%

barks
    at         100.00%

both
    a          100.00%

cat
    .          25.00%
    and        12.50%
    is         37.50%
    runs       12.50%
    sat        12.50%

dog
    .          25.00%
    are        25.00%
    barks      25.00%
    is         25.00%

every
    animal     50.00%
    cat        50.00%

is
    also       20.00%
    an         40.00%
    never      20.00%
    not        20.00%

mat
    .          100.00%

never
    a          100.00%

not
    a          100.00%

on
    the        100.00%

runs
    away       100.00%

sat
    on         100.00%

the
    cat        60.00%
    

## Bigram model output

The following code generates a set number of sentences using a bigram model trained on the provided data.

In [9]:
words = []
sentenceCounter = 0

while (sentenceCounter < NumberOfOutputSentences):
    currentWord = words[-1] if len(words) > 0 else '.'

    if currentWord not in bigrams: break
    
    currentWord = random.choices(list(bigrams[currentWord].keys()), weights=bigrams[currentWord].values())[0]
    if currentWord == ".": sentenceCounter += 1

    words.append(currentWord)

prettyPrint(words)

Every animal is also an animal. Every cat. The mat. Every animal. A cat and a dog barks at the cat is also an animal is never a dog.


# Generic code for n-Gram models
The following below can be reused for n-gram models with different values of n.

The output below shows a trigram data structure.

In [10]:
def generate_ngrams(n):
    ngrams = {}

    for i in range(len(tokenizedData) - n + 1):
        key = tuple(tokenizedData[i:i+n-1])
        nextWord = tokenizedData[i+n-1]

        if not key in ngrams:
            ngrams[key] = {nextWord: 1}
        elif nextWord not in ngrams[key]:
            ngrams[key][nextWord] = 1
        else:
            ngrams[key][nextWord] += 1

    return ngrams

ngrams = generate_ngrams(3)
print(f"{ngrams = }")


ngrams = {('a', 'cat'): {'is': 2, 'and': 1, '.': 1}, ('cat', 'is'): {'an': 2, 'never': 1}, ('is', 'an'): {'animal': 2}, ('an', 'animal'): {'.': 3}, ('animal', '.'): {'a': 1, 'both': 1, 'every': 1}, ('.', 'a'): {'dog': 1, 'cat': 1}, ('a', 'dog'): {'is': 1, 'are': 1, '.': 1}, ('dog', 'is'): {'also': 1}, ('is', 'also'): {'an': 1}, ('also', 'an'): {'animal': 1}, ('.', 'both'): {'a': 1}, ('both', 'a'): {'cat': 1}, ('cat', 'and'): {'a': 1}, ('and', 'a'): {'dog': 1}, ('dog', 'are'): {'animals': 1}, ('are', 'animals'): {'.': 1}, ('animals', '.'): {'every': 1}, ('.', 'every'): {'cat': 1, 'animal': 1}, ('every', 'cat'): {'is': 1}, ('every', 'animal'): {'is': 1}, ('animal', 'is'): {'not': 1}, ('is', 'not'): {'a': 1}, ('not', 'a'): {'cat': 1}, ('cat', '.'): {'a': 1, 'the': 1}, ('is', 'never'): {'a': 1}, ('never', 'a'): {'dog': 1}, ('dog', '.'): {'the': 1}, ('.', 'the'): {'cat': 2, 'dog': 1}, ('the', 'cat'): {'sat': 1, '.': 1, 'runs': 1}, ('cat', 'sat'): {'on': 1}, ('sat', 'on'): {'the': 1}, ('on',

## Print probabilities

In [11]:
print_ngram_probabilities(ngrams)

('.', 'a')
    cat        50.00%
    dog        50.00%

('.', 'both')
    a          100.00%

('.', 'every')
    animal     50.00%
    cat        50.00%

('.', 'the')
    cat        66.67%
    dog        33.33%

('a', 'cat')
    .          25.00%
    and        25.00%
    is         50.00%

('a', 'dog')
    .          33.33%
    are        33.33%
    is         33.33%

('also', 'an')
    animal     100.00%

('an', 'animal')
    .          100.00%

('and', 'a')
    dog        100.00%

('animal', '.')
    a          33.33%
    both       33.33%
    every      33.33%

('animal', 'is')
    not        100.00%

('animals', '.')
    every      100.00%

('are', 'animals')
    .          100.00%

('at', 'the')
    cat        100.00%

('barks', 'at')
    the        100.00%

('both', 'a')
    cat        100.00%

('cat', '.')
    a          50.00%
    the        50.00%

('cat', 'and')
    a          100.00%

('cat', 'is')
    an         66.67%
    never      33.33%

('cat', 'runs')
    away       

## Compare outputs for different n-values

In [66]:
def generate_text(ngrams):
    contextLength = len(next(iter(ngrams)))
    title = ['Unigram', 'Bigram', 'Trigram', 'Four-gram', 'Five-gram']

    BOLD = "\033[1m"
    RESET = "\033[0m"

    if contextLength < len(title):
        print(f'{BOLD}{title[contextLength]}{RESET}')
    else:
        print(f'{BOLD}{contextLength + 1}-gram{RESET}')

    startKeys = [key for key in ngrams.keys() if key[0] == '.']
    words = list(random.choice(startKeys))
    sentenceCounter = 0

    while (sentenceCounter < NumberOfOutputSentences):
        previousWords = tuple(words[-contextLength:])

        if previousWords not in ngrams: break
        
        currentWord = random.choices(list(ngrams[previousWords].keys()), weights=ngrams[previousWords].values())[0]
        if currentWord == ".": sentenceCounter += 1

        words.append(currentWord)

    words = words[1:]

    prettyPrint(words)
    print()

bigrams = generate_ngrams(2)
trigrams = generate_ngrams(3)
fourgrams = generate_ngrams(4)

generate_text(bigrams)
generate_text(trigrams)
generate_text(fourgrams)


Bigram
Every animal. A dog. The mat. A cat is never a cat is also an animal. The cat runs away.

Trigram
Both a cat is an animal. Every cat is an animal. A cat and a dog are animals. Every animal is not a cat is an animal. Both a cat is an animal.

Four-gram
The dog barks at the cat. The cat sat on the mat. The dog barks at the cat. The cat sat on the mat. The dog barks at the cat.

